### Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd

import altair as alt

from covidviz import data

In [ ]:
DATA_PATH = Path.cwd() / "data"
MIN_N_CASES = 50
day_counter = f"days_since_{MIN_N_CASES}_cases"

### Data preparation

#### Infections

In [ ]:
# Unfortunately, the mopo data seems to be refreshed only once a week at most
# mopo = pd.read_csv(
#     "https://interaktiv.morgenpost.de/corona-virus-karte-infektionen-deutschland-weltweit/data/Coronavirus.history.v2.csv"
# ).query("parent == 'Deutschland'")
# mopo["date"].max()

In [ ]:
# We'll use the RKI data instead
df = data.get_data(
    out_path=DATA_PATH,
    states=pd.read_csv(DATA_PATH / "bundeslaender.csv")["Bundesland"].unique(),
)

In [ ]:
daily_infections = df.pipe(data.prepare_daily_infections, n_cases=MIN_N_CASES).pipe(
    data.add_measures, measures=data.read_measure_data(DATA_PATH)
)

In [ ]:
daily_infections.head(2)

In [ ]:
daily_infections["Meldedatum"].max()

#### Google's mobility reports

In [ ]:
mobility = (
    pd.read_csv(
        DATA_PATH / "Global_Mobility_Report.csv",
        low_memory=False,
        parse_dates=["date"],
        infer_datetime_format=True,
    )
    .query("country_region_code == 'DE' and not sub_region_1.isna()")
    .rename(columns={"sub_region_1": "Bundesland", "date": "Meldedatum"})
    .drop(columns=["country_region_code", "country_region", "sub_region_2"])
)

In [ ]:
# change_cols_mapper = {
#     c: c[: c.find("percent_change") - 1] for c in mobility if "percent_change" in c
# }
# mobility_plot = (
#     mobility.melt(
#         id_vars=["sub_region_1", "date"],
#         value_vars=list(change_cols_mapper.keys()),
#         var_name="mobility_category",
#         value_name="mobility_percent_change",
#     )
#     .assign(
#         mobility_category=lambda df: df["mobility_category"].map(change_cols_mapper)
#     )
#     .rename(columns={"sub_region_1": "Bundesland", "date": "Meldedatum"})
# )

### Visualization

In [ ]:
percentage_change_cols_mapper = {c: c[: c.find("percent_change") - 1].replace("_", " ").title() for c in mobility.columns if "percent_change" in c}

plot_df = (
    daily_infections.assign(
        **{
            "absolute_growth": lambda df: df.groupby("Bundesland")[
                "infections_cumulative"
            ].transform(lambda s: s.diff()),
            "cases_logratio": lambda df: df.groupby("Bundesland")[
                "infections_cumulative"
            ].transform(lambda s: np.log(s).diff()),
            "Anzahl Maßnahmen": lambda df: df["Maßnahmen"].apply(
                lambda x: len(x.split("und")) if isinstance(x, str) else 0
            ),
        }
    )
    .query(f"{day_counter} >= 0")
    .merge(
        mobility,
        on=["Bundesland", "Meldedatum"],
        how="left",
    )
    .rename(columns=percentage_change_cols_mapper)
)

In [ ]:
plot_df.head()

In [ ]:
X_VARIABLE = "days_since_50_cases"
Y_VARIABLE = "daily_increase"

if Y_VARIABLE == "daily_increase":
    expression = "pow(E, datum.cases_logratio) - 1"
    y_title = "Daily Increase in Cumulative Cases"
    y_format = "%"
    y_domain = (0, 0.9)
    measure_level = "0.85"
    title = "Daily Increase of COVID-19 Cases in German States"
elif Y_VARIABLE == "doubling_time":
    expression = "log(2) / datum.cases_logratio"
    y_title = "Doubling Time (Days)"
    y_format = ""
    y_domain = (50, 0)
    measure_level = "45"
    title = "Doubling Time of COVID-19 Cases in German States"
elif Y_VARIABLE == "absolute_growth":
    raise NotImplementedError("This doesn't work yet.")
    expression = "datum.absolute_increase"
    y_title = "Absolute Growth in Cumulative Cases"
    y_format = ""
    title = "Absolute Growth of COVID-19 Cases in German States"
else:
    raise NotImplementedError(f"y variable {Y_VARIABLE} is not implemented.")

combined_charts = []
line_charts = []
for state in plot_df["Bundesland"].unique():
    base = alt.Chart(plot_df.query(f"Bundesland == '{state}'"), title=state).encode(
        x=alt.X(
            X_VARIABLE,
            axis=alt.Axis(title=X_VARIABLE.replace("_", " ").title(), offset=5),
        ),
        y=alt.Y("cases_logratio:Q"),
    )
    points = (
        base.transform_calculate(as_=Y_VARIABLE, calculate=expression)
        .mark_point()
        .encode(
            y=alt.Y(
                f"{Y_VARIABLE}:Q",
                scale=alt.Scale(domain=y_domain),
                axis=alt.Axis(format=y_format, title=y_title),
            ),
            color="Bundesland:N",
        )
    )
    measure_points = (
        base.mark_point(size=300, shape="diamond", color="grey", fill=None)
        .transform_calculate(y_level=measure_level)
        .encode(
            y="y_level:Q",
            size=alt.Size("Anzahl Maßnahmen:Q"),
            tooltip=["Meldedatum", "Maßnahmen"],
        )
        .interactive()
    )
    lines = (
        points.transform_loess(
            on=X_VARIABLE,
            loess=Y_VARIABLE,
            as_=[X_VARIABLE, f"{Y_VARIABLE}_loess"],
            groupby=["Bundesland"],
        )
        .mark_line()
        .encode(
            y=alt.Y(
                f"{Y_VARIABLE}_loess:Q",
                scale=alt.Scale(domain=y_domain),
                axis=alt.Axis(format=y_format, title=y_title),
            ),
            tooltip=[X_VARIABLE],
        )
    )
    line_charts.append(lines.properties(width=900, height=300, title=title))
    combined_charts.append(
        (points + measure_points + lines).properties(width=900, height=300)
    )

In [ ]:
alt.layer(*line_charts)

In [ ]:
alt.vconcat(*combined_charts)

#### Absolute growth

In [ ]:
X_VARIABLE = "days_since_50_cases"
Y_VARIABLE = "absolute_growth"
y_title = "Absolute Growth in Cumulative Cases"
y_format = ""
title = "Absolute Growth of COVID-19 Cases in German States"

In [ ]:
combined_charts = []
line_charts = []
for state in plot_df["Bundesland"].unique():

    base = alt.Chart(plot_df.query(f"Bundesland == '{state}'"), title=state).encode(
        x=alt.X(
            X_VARIABLE, axis=alt.Axis(title=X_VARIABLE.replace("_", " ").title(), offset=5)
        ),
        y=alt.Y(f"{Y_VARIABLE}:Q"),
    )
    points = base.mark_point().encode(y=alt.Y(f"{Y_VARIABLE}:Q"), color="Bundesland:N")
    measure_points = (
        base.mark_point(size=600, shape="diamond", color="grey", fill=None)
        .transform_calculate(y_level="0")
        .encode(
            y="y_level:Q",
            size=alt.Size("Anzahl Maßnahmen:Q"),
            tooltip=["Meldedatum", "Maßnahmen"],
        )
        .interactive()
    )
    lines = (
        points.transform_loess(
            on=X_VARIABLE,
            loess=Y_VARIABLE,
            as_=[X_VARIABLE, f"{Y_VARIABLE}_loess"],
            groupby=["Bundesland"],
        )
        .mark_line()
        .encode(
            y=alt.Y(f"{Y_VARIABLE}_loess:Q", axis=alt.Axis(format="", title=y_title)),
            tooltip=[X_VARIABLE],
        )
    )
    combined_charts.append(
        (points + measure_points + lines).properties(width=900, height=300)
    )

In [ ]:
alt.vconcat(*combined_charts)

##### Adding the mobility data

In [ ]:
state = "Baden-Württemberg"
base = alt.Chart(plot_df.query(f"Bundesland == '{state}'"), title=state).encode(
    x=alt.X(
        X_VARIABLE, axis=alt.Axis(title=X_VARIABLE.replace("_", " ").title(), offset=5)
    ),
    y=alt.Y(f"{Y_VARIABLE}:Q"),
)
points = base.mark_point().encode(y=alt.Y(f"{Y_VARIABLE}:Q"), color="Bundesland:N")
measure_points = (
    base.mark_point(size=600, shape="diamond", color="grey", fill=None)
    .transform_calculate(y_level="0")
    .encode(
        y="y_level:Q",
        size=alt.Size("Anzahl Maßnahmen:Q"),
        tooltip=["Meldedatum", "Maßnahmen"],
    )
    .interactive()
)
lines = (
    points.transform_loess(
        on=X_VARIABLE,
        loess=Y_VARIABLE,
        as_=[X_VARIABLE, f"{Y_VARIABLE}_loess"],
        groupby=["Bundesland"],
    )
    .mark_line()
    .encode(
        y=alt.Y(f"{Y_VARIABLE}_loess:Q", axis=alt.Axis(format="", title=y_title)),
        tooltip=[X_VARIABLE],
    )
)
activity = (
    base.mark_area()
    .transform_fold(
        fold=list(percentage_change_cols_mapper.values()),
        as_=["Mobility Category", "mobility_change_percent"],
    )
    .transform_calculate(
        as_="Mobility Change", calculate="datum.mobility_change_percent / 100"
    )
    .encode(
        y=alt.Y("Mobility Change:Q", axis=alt.Axis(format="%")),
        color="Mobility Category:N",
    )
)

In [ ]:
alt.layer((points + measure_points + lines), activity).resolve_scale(
    y="independent"
).properties(width=900, height=300)

### Experiments

In [ ]:
# day_markers = (
#     base.transform_calculate(
#         as_="doubling_time", calculate="log(2) / datum.cases_logratio"
#     )
#     .mark_point(clip=True, color="green")
#     .encode(
#         y=alt.Y(
#             "doubling_time:Q",
#             scale=alt.Scale(domain=(21, 0)),
#             axis=alt.Axis(
#                 title="Doubling Time (Days)",
#                 grid=True,
#                 gridColor="grey",
#                 gridWidth=0.75,
#                 orient="right",
#                 values=[2, 3, 4, 5, 6, 7, 14, 21],
#             ),
#         )
#     )
# )
# day_markers